1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때 , 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [3]:
%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax_with_markdown.docx")
docs = loader.load_and_split(text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-index"
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

db = PineconeVectorStore.from_documents(
    docs,
    embeddings,
    index_name=index_name
)

/Users/kdh/Documents/workspace/LLM/llm-app/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# retrieved_docs = db.similarity_search(query, k=3)

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [ ]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [6]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

In [12]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [7]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
ai_message = qa_chain.invoke(query)

In [9]:
ai_message

'주어진 자료만으로는 정확한 소득세를 알 수 없습니다. 근로소득공제·인적공제·4대보험·연말정산 항목 등으로 과세표준이 달라지고, 소득세는 누진세율을 과세표준에 따라 적용하기 때문에 과세표준 정보가 필요합니다. 원하시면 연봉 5천만원에서 국민연금·건강보험 등 공제액과 부양가족 수 등을 알려주시면 대략의 세액을 계산해 드리겠습니다.'